In [23]:
#When F1=700

get_ipython().run_line_magic('reset', '-f')

import numpy as np
import pandas
import sympy
from termcolor import colored


###### inputs

stream_direction = ["in","in","out","out"] # directions of streams
component_labels = ["Flow rate","Methene","Ethene","Propyne","Nitrogen","Oxygen","CO2"] #chemical components
fraction_prefix = "y" # mass (x) or mole (y) fraction

###### process relation

process_equation = ["F1=0.25*F2","F3=0.5*F4","y2N=0.1*y3P"] ## eg:"F1=0.25*F2"

###### input fraction , flow rate

F1=700
#y1M=0.33
y1E=0.29
#y2N=0.79
y3M=0.1
#y3P=0.3
y4E=0.06
y4O=0.07

y1N=0; y1O=0; y2M=0; y2E=0; y2P=0; y2C=0; y3N=0; y3O=0; y4P=0; y4C=0;

######


Ns = len(stream_direction)
Nc = len(component_labels) - 1 
Np = len(process_equation)



stream_labels = []
for i in range(Ns):
    stream_labels.append("stream " + str(i+1) + "-" + stream_direction[i])


V_empty = np.empty([Ns, Nc+1],dtype=object)  


df = pandas.DataFrame(V_empty, index=stream_labels, columns=component_labels)
print (df)


all_variables = []


for i in range(len(stream_direction)):
    all_variables.append(component_labels[0][0] + str(i+1))


for i in range(len(component_labels[1:])): 
    for j in range(len(stream_direction)):
        all_variables.append(fraction_prefix + str(j+1) 
                             + component_labels[i+1][0])
        


all_symbols = sympy.symbols(all_variables)




V_initial = V_empty.copy()

given_variables = []

unknown_index=[]
for i in range(len(all_variables)):
    
 
    index_row = int(all_variables[i][1]) - 1
    
    if all_variables[i][0] == component_labels[0][0]:
        index_column = 0
    
    else:
        for j in range(len(component_labels)):
            
            if all_variables[i][2] == component_labels[j][0]:
                index_column = j
    
    if all_variables[i] in globals():

        V_initial[index_row, index_column] = eval(all_variables[i])
        given_variables.append(all_variables[i])
    else:
        V_initial[index_row, index_column] = all_symbols[i]
        unknown_index.append([index_row,index_column])


print("\n")

 
df = pandas.DataFrame(V_initial, index=stream_labels, columns=component_labels)
print (df)   


N_zeros = np.count_nonzero(V_initial == 0) 
Nv = Ns*(Nc+1) - N_zeros + Np            
Nd = Nv - Ns - Nc -Np                   


print ("\nThe number of total variables (zeros not included):", Nv) 
print (colored("The number of design variables:",attrs=["bold"]), Nd)
print ("\nYou have given", len(given_variables)-N_zeros+Np, "variables")



if Np != 0:
    print("\nThere are",Np,"process relations")
    for i in process_equation:
        print(i)


# Check if you have given too many or didn't give enough design variables
if len(given_variables)-N_zeros+Np > Nd:
    print (colored("You have given too many input variables","red"))
if len(given_variables)-N_zeros+Np < Nd:
    print (colored("You didn't give enough input variables","red"))
    
    
u=0
for i in range(len(stream_direction)):
    if all_variables[i] not in globals():
        u+=1
    if u==len(stream_direction):
        print(colored("\nYou didn't provide any Flow information so can't solve the problem","red"))

###########################################################################################################################    
    
sym_equations=[]

if Np!= 0:
    function=process_equation
    for i in range(len(process_equation)):
        temp_eq=[]
        n=0
        for j in range(len(process_equation[i])):
            if process_equation[i][j] == "=":
                n=1
            if n>0:
                temp_eq.append("-("+process_equation[i][j+1:]+")")
                break
            if n==0:
                temp_eq.append(process_equation[i][j])
        function[i]="".join(temp_eq)

for i in range(len(process_equation)):
    sym_equations.append(sympy.Eq((sympy.sympify(function[i])),0))
    

###########################################################################################################################    
   
unknown_variables = [element for element in all_variables if element not in given_variables]
number_unknowns = len(unknown_variables)
print ("\nThere are", number_unknowns, "unknown variables:")
print (*unknown_variables,sep=" ,")

signed_V = V_initial.copy()

for i in range(signed_V.shape[0]):
    if stream_direction[i] == "out":
        signed_V[i,0] = signed_V[i,0]*-1
        
functions = []
        
for i in range(Nc):
    functions.append(sum(signed_V[:,0]*signed_V[:,i+1]))
    
for i in range(Ns):
    functions.append(sum(signed_V[i,1:]) - 1)   
    

for i in functions:
    sym_equations.append(sympy.Eq(i,0))
    

solutions = sympy.solve(sym_equations,unknown_variables)

solutions = eval(str(solutions))


Final_V = V_initial.copy()

if len(solutions)!=0:

    print("\nThe solutions are:")

    for i in range(len(unknown_variables)):
        print(unknown_variables[i],"=",sympy.Float(solutions[0][i],5))
    
    for i in range(len(unknown_variables)):
        Final_V[unknown_index[i][0],unknown_index[i][1]] =sympy.Float(solutions[0][i],5)

    for i in range(len(solutions[0])):
        if int(solutions[0][i])<0:
            print(colored("\nWarning !! The solution contains negative numbers","red"))
            break

elif len(solutions)==0 and len(given_variables)-N_zeros+Np == Nd:
    
    print(colored("\nWarning there should be existing dependent equations","red",attrs=["bold"]))

print ("\n")    


df = pandas.DataFrame(Final_V, index=stream_labels, columns=component_labels)
df


             Flow rate Methene Ethene Propyne Nitrogen Oxygen   CO2
stream 1-in       None    None   None    None     None   None  None
stream 2-in       None    None   None    None     None   None  None
stream 3-out      None    None   None    None     None   None  None
stream 4-out      None    None   None    None     None   None  None


             Flow rate Methene Ethene Propyne Nitrogen Oxygen  CO2
stream 1-in        700     y1M   0.29     y1P        0      0  y1C
stream 2-in         F2       0      0       0      y2N    y2O    0
stream 3-out        F3     0.1    y3E     y3P        0      0  y3C
stream 4-out        F4     y4M   0.06       0      y4N   0.07    0

The number of total variables (zeros not included): 21
The number of design variables: 8

You have given 8 variables

There are 3 process relations
F1=0.25*F2
F3=0.5*F4
y2N=0.1*y3P

There are 13 unknown variables:
F2 ,F3 ,F4 ,y1M ,y4M ,y3E ,y1P ,y3P ,y2N ,y4N ,y2O ,y1C ,y3C

The solutions are:
F2 = 2800.0
F3 = 1166.7
F4 

,Flow rate,Methene,Ethene,Propyne,Nitrogen,Oxygen,CO2
stream 1-in,700,-0.70000,0.29,15.694,0,0,-14.284
stream 2-in,2800.0,0,0,0,0.94167,0.058333,0
stream 3-out,1166.7,0.1,0.054000,9.4167,0,0,-8.5707
stream 4-out,2333.3,-0.26000,0.06,0,1.1300,0.07,0


In [13]:
solutions

[]